# Hyperlane Core Deployment Analysis - Sepolia, Pruv & Arbitrum Sepolia Networks

This notebook analyzes the gas costs and transaction patterns for Hyperlane core deployment on Sepolia, Pruv, and Arbitrum Sepolia networks.

In [1]:
import json
import pandas as pd
import re
from datetime import datetime
from IPython.display import display

# Define chains for processing
chains = ['arbitrumsepolia', 'pruv', 'sepolia']

## 1. Load and Parse Data

In [2]:
# Load transaction data for all chains
chain_data = {}
chain_dfs = {}

for chain in chains:
    print(f'=== LOADING {chain.upper()} DATA ===')
    with open(f'data/core_deployment/raw/{chain}_txs.json', 'r') as f:
        chain_data[chain] = json.load(f)
    
    print(f'Total transactions in dataset: {len(chain_data[chain]["result"])}')
    
    # Convert to DataFrame
    chain_dfs[chain] = pd.DataFrame(chain_data[chain]['result'])
    
    # Display basic info
    print(f'DataFrame shape: {chain_dfs[chain].shape}')
    print(f'Columns: {list(chain_dfs[chain].columns)}')
    print()

# Create individual variables for backward compatibility
sepolia_data = chain_data['sepolia']
sepolia_df = chain_dfs['sepolia']
pruv_data = chain_data['pruv']
pruv_df = chain_dfs['pruv']
arbitrumsepolia_data = chain_data['arbitrumsepolia']
arbitrumsepolia_df = chain_dfs['arbitrumsepolia']

=== LOADING ARBITRUMSEPOLIA DATA ===
Total transactions in dataset: 123
DataFrame shape: (123, 18)
Columns: ['blockHash', 'blockNumber', 'confirmations', 'contractAddress', 'cumulativeGasUsed', 'from', 'gas', 'gasPrice', 'gasUsed', 'hash', 'input', 'isError', 'nonce', 'timeStamp', 'to', 'transactionIndex', 'txreceipt_status', 'value']

=== LOADING PRUV DATA ===
Total transactions in dataset: 511
DataFrame shape: (511, 18)
Columns: ['blockHash', 'blockNumber', 'confirmations', 'contractAddress', 'cumulativeGasUsed', 'from', 'gas', 'gasPrice', 'gasUsed', 'hash', 'input', 'isError', 'nonce', 'timeStamp', 'to', 'transactionIndex', 'txreceipt_status', 'value']

=== LOADING SEPOLIA DATA ===
Total transactions in dataset: 1025
DataFrame shape: (1025, 18)
Columns: ['blockHash', 'blockNumber', 'confirmations', 'contractAddress', 'cumulativeGasUsed', 'from', 'gas', 'gasPrice', 'gasUsed', 'hash', 'input', 'isError', 'nonce', 'timeStamp', 'to', 'transactionIndex', 'txreceipt_status', 'value']



## 2. Extract Deployment Start Timestamps

In [3]:
# Extract deployment timestamps for all chains
deployment_timestamps = {}
deployment_timestamps_ms = {}
deployment_dates = {}

for chain in chains:
    print(f'=== {chain.upper()} DEPLOYMENT TIMESTAMP ===')
    with open(f'data/core_deployment/raw/{chain}_deployment_cli_output.txt', 'r') as f:
        cli_output = f.read()
    
    # Extract timestamp using regex
    timestamp_match = re.search(r'"time":(\d{13})', cli_output)
    
    if timestamp_match:
        deployment_timestamps_ms[chain] = int(timestamp_match.group(1))
        deployment_timestamps[chain] = deployment_timestamps_ms[chain] / 1000
        deployment_dates[chain] = datetime.fromtimestamp(deployment_timestamps[chain])
        print(f'Deployment start timestamp: {deployment_timestamps_ms[chain]}')
        print(f'Deployment start date: {deployment_dates[chain]}')
    else:
        print(f'Timestamp not found in {chain} CLI output')
    print()

# Create individual variables for backward compatibility
sepolia_deployment_timestamp_ms = deployment_timestamps_ms['sepolia']
sepolia_deployment_timestamp = deployment_timestamps['sepolia']
sepolia_deployment_date = deployment_dates['sepolia']
pruv_deployment_timestamp_ms = deployment_timestamps_ms['pruv']
pruv_deployment_timestamp = deployment_timestamps['pruv']
pruv_deployment_date = deployment_dates['pruv']
arbitrumsepolia_deployment_timestamp_ms = deployment_timestamps_ms['arbitrumsepolia']
arbitrumsepolia_deployment_timestamp = deployment_timestamps['arbitrumsepolia']
arbitrumsepolia_deployment_date = deployment_dates['arbitrumsepolia']

=== ARBITRUMSEPOLIA DEPLOYMENT TIMESTAMP ===
Deployment start timestamp: 1754923881680
Deployment start date: 2025-08-11 21:51:21.680000

=== PRUV DEPLOYMENT TIMESTAMP ===
Deployment start timestamp: 1754885795563
Deployment start date: 2025-08-11 11:16:35.563000

=== SEPOLIA DEPLOYMENT TIMESTAMP ===
Deployment start timestamp: 1754887631358
Deployment start date: 2025-08-11 11:47:11.358000



## 3. Filter and Process Transactions

In [4]:
# Process transactions for all chains
deployment_txs = {}

for chain in chains:
    print(f'=== {chain.upper()} TRANSACTION FILTERING ===')
    
    # Get the dataframe and timestamp for this chain
    df = chain_dfs[chain]
    timestamp = deployment_timestamps[chain]
    
    # Convert timestamp and datetime columns
    df['timeStamp'] = pd.to_numeric(df['timeStamp'])
    df['datetime'] = pd.to_datetime(df['timeStamp'], unit='s')
    
    # Filter transactions from deployment start
    deployment_txs[chain] = df[df['timeStamp'] >= timestamp].copy()
    
    # Convert gas and value columns to numeric
    deployment_txs[chain]['gasUsed'] = pd.to_numeric(deployment_txs[chain]['gasUsed'])
    deployment_txs[chain]['gasPrice'] = pd.to_numeric(deployment_txs[chain]['gasPrice'])
    deployment_txs[chain]['value'] = pd.to_numeric(deployment_txs[chain]['value'])
    
    # Calculate gas cost in ETH
    deployment_txs[chain]['gasCostWei'] = deployment_txs[chain]['gasUsed'] * deployment_txs[chain]['gasPrice']
    deployment_txs[chain]['gasCostETH'] = deployment_txs[chain]['gasCostWei'] / 1e18
    
    # Identify transaction types
    deployment_txs[chain]['isContractDeployment'] = deployment_txs[chain]['to'].isna() | (deployment_txs[chain]['to'] == '')
    
    # Print statistics
    print(f'Deployment transactions: {len(deployment_txs[chain])}')
    
    if len(deployment_txs[chain]) > 0:
        print(f'\nFirst transaction: {deployment_txs[chain].iloc[0]["datetime"]}')
        print(f'Last transaction: {deployment_txs[chain].iloc[-1]["datetime"]}')
    
    print()

# Create individual variables for backward compatibility
sepolia_deployment_txs = deployment_txs['sepolia']
pruv_deployment_txs = deployment_txs['pruv']
arbitrumsepolia_deployment_txs = deployment_txs['arbitrumsepolia']

=== ARBITRUMSEPOLIA TRANSACTION FILTERING ===
Deployment transactions: 30

First transaction: 2025-08-11 14:56:04
Last transaction: 2025-08-11 14:51:26

=== PRUV TRANSACTION FILTERING ===
Deployment transactions: 30

First transaction: 2025-08-11 04:18:45
Last transaction: 2025-08-11 04:16:39

=== SEPOLIA TRANSACTION FILTERING ===
Deployment transactions: 30

First transaction: 2025-08-11 04:58:00
Last transaction: 2025-08-11 04:47:24



## 4. Detailed Transaction Tables

In [5]:
# Create detailed transaction tables for all chains
for chain in chains:
    print(f'=== {chain.upper()} DETAILED TRANSACTION TABLE ===')
    print()
    
    deployment_txs = globals()[f'{chain}_deployment_txs']
    
    if len(deployment_txs) > 0:
        # Create display table
        display_data = []
        
        for idx, row in deployment_txs.iterrows():
            context = 'Contract Deployment' if row['isContractDeployment'] else 'Contract Call'
            
            display_data.append({
                'Timestamp': row['datetime'].strftime('%Y-%m-%d %H:%M:%S'),
                'TX Hash': row['hash'],
                'Context': context,
                'Gas Limit': int(row['gas']),
                'Gas Used': int(row['gasUsed']),
                'Gas Price (Wei)': int(row['gasPrice']),
                'TX Fee (ETH)': f"{row['gasCostETH']:.8f}"
            })
        
        display_df = pd.DataFrame(display_data)
        display(display_df)
        
        # Create individual variables for backward compatibility
        globals()[f'{chain}_display_data'] = display_data
        globals()[f'{chain}_display_df'] = display_df
    else:
        print(f'No {chain} deployment transactions found.')
    
    print()
    print()

=== ARBITRUMSEPOLIA DETAILED TRANSACTION TABLE ===



,Timestamp,TX Hash,Context,Gas Limit,Gas Used,Gas Price (Wei),TX Fee (ETH)
0,2025-08-11 14:56:04,0x0a2f5b19516bfdc263691467a8547f798b304599682d...,Contract Call,51024,46027,100000000,0.00000460
1,2025-08-11 14:55:47,0x36cca0cfc8f94509be9446b1ae1a6572bb6bf16f85f0...,Contract Deployment,717997,647666,100000000,0.00006477
2,2025-08-11 14:55:31,0x4f16d11253b1073bf16142bf2194d1d67886e4e43a10...,Contract Deployment,1587875,1432332,100000000,0.00014323
3,2025-08-11 14:55:17,0xa8dcffc30ee8f36f320f4ca15424b6196117b070be9d...,Contract Deployment,1459762,1316770,100000000,0.00013168
4,2025-08-11 14:55:01,0x1778d20b73b0c036beb9ec4929fad20f515d1a8e37fb...,Contract Call,635985,573689,100000000,0.00005737
5,2025-08-11 14:54:57,0xdf5c3be7bf1aaf8e45eb9ca283b6bb027b59cdd6b469...,Contract Deployment,4333160,3908697,100000000,0.00039087
6,2025-08-11 14:54:41,0x9badc27e3f5e02efeb5427ec64d8ad48cf9a965dd209...,Contract Deployment,598156,539564,100000000,0.00005396
7,2025-08-11 14:54:27,0xaa997479e503dff9d29242add6ac2a49d1fd0103a886...,Contract Deployment,403871,364310,100000000,0.00003643
8,2025-08-11 14:54:25,0x206ff881b2e0503f2c732a648e6500d1d3646cdc663f...,Contract Call,162205,158492,100000000,0.00001585
9,2025-08-11 14:54:22,0xe091553a3112ed72379056867f7d3e27f2c6fe841829...,Contract Call,39561,35687,100000000,0.00000357




=== PRUV DETAILED TRANSACTION TABLE ===



,Timestamp,TX Hash,Context,Gas Limit,Gas Used,Gas Price (Wei),TX Fee (ETH)
0,2025-08-11 04:18:45,0x4fddaf27e1ccfddca236b8adfcb04459a5d2bd674131...,Contract Call,50629,46027,0,0.00000000
1,2025-08-11 04:18:41,0x7e2b5f62fd34a2c522629593995972bb19a203390534...,Contract Deployment,712231,647483,0,0.00000000
2,2025-08-11 04:18:37,0x68786cce2e79eef7a2e7092aaac23283429c3cab3050...,Contract Deployment,1575077,1431889,0,0.00000000
3,2025-08-11 04:18:31,0x25e9fceef63b83d0d3a5891f754e2974cc4a1bfabf89...,Contract Deployment,1447961,1316329,0,0.00000000
4,2025-08-11 04:18:27,0xcd8151f2f9e0362e4fb5ba726b4643335e9cb52be959...,Contract Call,632149,573548,0,0.00000000
5,2025-08-11 04:18:23,0x10e60891c02625224c1c67a2cf257ab6f392f0387dbf...,Contract Deployment,4298252,3907502,0,0.00000000
6,2025-08-11 04:18:19,0xe12dbabc976ac64e9aabcfb532ecba6a0e08b8260ca8...,Contract Deployment,593354,539413,0,0.00000000
7,2025-08-11 04:18:15,0xf4dfa26323c4de3c5403dbf27fc640db48c86a27ae3b...,Contract Deployment,400622,364202,0,0.00000000
8,2025-08-11 04:18:09,0xd0641878f70e4cdac9205a5e98c246a72c715d9cd4f9...,Contract Call,161627,158479,0,0.00000000
9,2025-08-11 04:18:07,0x18fe504600420ef62c4d9e21355ef8bb437cd447fe03...,Contract Call,39338,35687,0,0.00000000




=== SEPOLIA DETAILED TRANSACTION TABLE ===



,Timestamp,TX Hash,Context,Gas Limit,Gas Used,Gas Price (Wei),TX Fee (ETH)
0,2025-08-11 04:58:00,0x336fdf79217f6687dee26d6a6f37a22b374dd74e45c5...,Contract Call,51049,46027,1002108,0.00000005
1,2025-08-11 04:57:48,0xb1cbc2529542c5df7838eb30e8d14f4a26bc5b324b1f...,Contract Deployment,719360,647657,1002199,0.00000065
2,2025-08-11 04:57:36,0x9263c99fd82cbd8b3b6e0c8cc93545422d4eac1727b6...,Contract Deployment,1589332,1432313,1001986,0.00000144
3,2025-08-11 04:57:24,0xf108da760db5bd6a13466d93d7e3109056f9633c8e15...,Contract Deployment,1461212,1316757,1002005,0.00000132
4,2025-08-11 04:56:48,0x820f277eb0b9b71a36834bfde453e5423fc76f379c80...,Contract Call,637348,573688,1002460,0.00000058
5,2025-08-11 04:56:36,0x7e692ece4b7751489e17fd554c27b039224148a6d656...,Contract Deployment,4334924,3908652,1002447,0.00000392
6,2025-08-11 04:56:24,0x49747ef4685f306a9faeceeb2ed0893c9a613a5f7d64...,Contract Deployment,599506,539557,1002251,0.00000054
7,2025-08-11 04:56:00,0x81b6f4a133ae412346bd3e3a0f122d6281f50c3da8d5...,Contract Deployment,405199,364304,1002269,0.00000037
8,2025-08-11 04:55:48,0xd5f9cf69457245d6281d6f26c97944c6245005170354...,Contract Call,163343,158491,1002261,0.00000016
9,2025-08-11 04:55:36,0x4a1f38a0868b9ff3a4f440f5d63d5707825eef85cdda...,Contract Call,39652,35687,1101825,0.00000004


## 5. Deployment Summary

In [6]:
# Create deployment summary comparison table
print('=== DEPLOYMENT SUMMARY COMPARISON ===')
print()

# Calculate metrics for all networks
sepolia_total_gas_limit = pd.to_numeric(sepolia_deployment_txs['gas']).sum()
sepolia_total_gas_used = sepolia_deployment_txs['gasUsed'].sum()
sepolia_total_fees_eth = sepolia_deployment_txs['gasCostETH'].sum()
sepolia_gas_efficiency = (sepolia_total_gas_used / sepolia_total_gas_limit) * 100
sepolia_avg_gas_price = sepolia_deployment_txs['gasPrice'].mean()

pruv_total_gas_limit = pd.to_numeric(pruv_deployment_txs['gas']).sum()
pruv_total_gas_used = pruv_deployment_txs['gasUsed'].sum()
pruv_total_fees_eth = pruv_deployment_txs['gasCostETH'].sum()
pruv_gas_efficiency = (pruv_total_gas_used / pruv_total_gas_limit) * 100
pruv_avg_gas_price = pruv_deployment_txs['gasPrice'].mean()

arbitrumsepolia_total_gas_limit = pd.to_numeric(arbitrumsepolia_deployment_txs['gas']).sum()
arbitrumsepolia_total_gas_used = arbitrumsepolia_deployment_txs['gasUsed'].sum()
arbitrumsepolia_total_fees_eth = arbitrumsepolia_deployment_txs['gasCostETH'].sum()
arbitrumsepolia_gas_efficiency = (arbitrumsepolia_total_gas_used / arbitrumsepolia_total_gas_limit) * 100
arbitrumsepolia_avg_gas_price = arbitrumsepolia_deployment_txs['gasPrice'].mean()

# Create comparison DataFrame
comparison_data = {
    'Network': ['Sepolia', 'Pruv', 'Arbitrum Sepolia'],
    'Total Gas Limit': [f'{sepolia_total_gas_limit:,}', f'{pruv_total_gas_limit:,}', f'{arbitrumsepolia_total_gas_limit:,}'],
    'Total Gas Used': [f'{sepolia_total_gas_used:,}', f'{pruv_total_gas_used:,}', f'{arbitrumsepolia_total_gas_used:,}'],
    'Gas Efficiency (%)': [f'{sepolia_gas_efficiency:.2f}%', f'{pruv_gas_efficiency:.2f}%', f'{arbitrumsepolia_gas_efficiency:.2f}%'],
    'Avg Gas Price (Wei)': [f'{sepolia_avg_gas_price:,.0f}', f'{pruv_avg_gas_price:,.0f}', f'{arbitrumsepolia_avg_gas_price:,.0f}'],
    'Total Fees (ETH)': [f'{sepolia_total_fees_eth:.6f}', f'{pruv_total_fees_eth:.6f}', f'{arbitrumsepolia_total_fees_eth:.6f}']
}

comparison_df = pd.DataFrame(comparison_data)
print('📊 NETWORK COMPARISON TABLE:')
print()
display(comparison_df)

print()
print('📈 KEY INSIGHTS:')
print(f'• Highest Gas Usage: {comparison_df.loc[comparison_df["Total Gas Used"].str.replace(",", "").astype(int).idxmax(), "Network"]}')
print(f'• Most Efficient: {comparison_df.loc[comparison_df["Gas Efficiency (%)"].str.replace("%", "").astype(float).idxmax(), "Network"]}')
print(f'• Lowest Fees: {comparison_df.loc[comparison_df["Total Fees (ETH)"].astype(float).idxmin(), "Network"]}')
print(f'• Highest Avg Gas Price: {comparison_df.loc[comparison_df["Avg Gas Price (Wei)"].str.replace(",", "").astype(float).idxmax(), "Network"]}')
print(f'• Lowest Avg Gas Price: {comparison_df.loc[comparison_df["Avg Gas Price (Wei)"].str.replace(",", "").astype(float).idxmin(), "Network"]}')

=== DEPLOYMENT SUMMARY COMPARISON ===

📊 NETWORK COMPARISON TABLE:



,Network,Total Gas Limit,Total Gas Used,Gas Efficiency (%),Avg Gas Price (Wei),Total Fees (ETH)
0,Sepolia,"29,552,244","26,638,844",90.14%,"1,044,889",0.000027
1,Pruv,"29,283,904","26,629,406",90.94%,0,0.000000
2,Arbitrum Sepolia,"29,521,603","26,639,196",90.24%,"100,000,000",0.002664



📈 KEY INSIGHTS:
• Highest Gas Usage: Arbitrum Sepolia
• Most Efficient: Pruv
• Lowest Fees: Pruv
• Highest Avg Gas Price: Arbitrum Sepolia
• Lowest Avg Gas Price: Pruv
